# L2: Information Extraction

In this lab you will implement and evaluate a simple system for information extraction. The task of the system is to read sentences and extract pairs of the form $(x, y)$, where $x$&nbsp;is a string denoting a person, $y$&nbsp;is a string denoting an organisation, and $x$ is the &lsquo;leader&rsquo; of&nbsp;$y$. Consider the following example sentence:

<blockquote>
Mr. Obama also selected Lisa Jackson to head the Environmental Protection Agency.
</blockquote>

From this sentence the system should extract the pair
```
("Lisa Jackson", "Environmental Protection Agency")
```

The system will have to solve the following sub-tasks:
* entity extraction &ndash; identifying mentions of person and organisation entities in text
* relation extraction &ndash; identifying instances of the &lsquo;is-leader-of&rsquo; relation

The data set for the lab consists of 62,010&nbsp;sentences from the [Groningen Meaning Bank](http://gmb.let.rug.nl) (release 2.2.0), an open corpus of English. To analyse the sentences you will use [spaCy](https://spacy.io/).

## Getting started

The first cell imports the Python module required for this lab.

In [1]:
import tm2

The next cell imports spaCy and loads its English language model.

In [2]:
import spacy

nlp = spacy.load('en')

## Data and gold standard

The data is contained in the following file:

In [3]:
data_file = "/home/TDDE16/labs/l2/data/gmb.txt"

The `tm2` module defines a function `read_data` that returns an iterator over the lines in a file. You should use this function to read the data for this lab. Use the optional argument `n` to restrict the iteration to the first few lines of the file. Here is an example:

In [4]:
for sentence in tm2.read_data(data_file, n=3):
    print(sentence)

Masked assailants with grenades and automatic weapons attacked a wedding party in southeastern Turkey, killing 45 people and wounding at least six others.
Turkish officials said the attack occurred Monday in the village of Bilge about 600 kilometers from Ankara.
The wounded were taken to the hospital in the nearby city of Mardin.


In addition to the raw data, we also provide you with a gold standard of entity pairs that your system should be able to extract. The following code loads these pairs from the file `gold.txt` and adds them to the set `gold`. Each pair is augmented with the identifier of the sentence (line number in the data file) which it was extracted from. Note that the sentence (line) numbering starts at index&nbsp;0.

In [5]:
gold_file = "/home/TDDE16/labs/l2/data/gold.txt"

gold = set()
with open(gold_file) as fp:
    for line in fp:
        columns = line.rstrip().split('\t')
        gold.add((int(columns[0]), columns[1], columns[2]))

The following code prints the 10&nbsp;first pairs from the gold standard:

In [31]:
for i, person, org in sorted(gold)[:10]:
    print("{}\t{}\t{}".format(i, person, org))
    

802	Ali Zardari	Pakistan People 's Party
2297	Abdul Aziz al-Hakim	Supreme Council
4823	Slavkov	Bulgarian National Olympic Committee
7902	Mr. Hakim	Supreme Council
8206	J. Patrick Boyle	American Meat Institute
8633	Ali Rodriguez	Petroleos de Venezuela
9004	Foreign Minister Joschka Fischer	Green Party
11021	Khalaf	al-Qaida
11259	Joseph Domenech	U.N. 's Food and Agricultural Organization
13043	David Petraeus	U.S. Central Command


You should take a moment to have a look at the data file and see these pairs in the context of the sentence they were extracted from.

## Entity extraction

To implement the entity extraction part of your system, you do not need to do much, as you can use the full natural language processing power built into spaCy. The following code extracts the entities from the first 5&nbsp;sentences of the data:

In [14]:
for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file, n=50))):
    
    for ent in doc.ents:
        print("{}\t{}\t{}\t{}".format(ent.text, ent.start, ent.end, ent.label_))

Turkey	13	14	GPE
45	16	17	CARDINAL
at least six	20	23	CARDINAL
Turkish	0	1	NORP
Monday	6	7	DATE
Bilge	11	12	ORG
about 600 kilometers	12	15	QUANTITY
Ankara	16	17	GPE
Mardin	12	13	ORG
Interior	0	1	ORG
Besir Atalay	2	4	PERSON
Turkey	0	1	GPE
NTV	2	3	ORG
Kurdish	21	22	NORP
The United Nations	0	3	ORG
Haiti	4	5	GPE
Tuesday	4	5	DATE
U.N.	6	7	ORG
Kofi Annan	10	12	PERSON
Haiti	13	14	GPE
first	26	27	ORDINAL
Jean-Bertrand Aristide	35	39	ORG
February	40	41	DATE
2004	42	43	DATE
recent months	8	10	DATE
November this year	4	7	DATE
December	18	19	DATE
Americans	11	12	NORP
46 million	6	8	CARDINAL
Americans	8	9	NORP
age this year	16	19	DATE
VOA	0	1	ORG
Mil Arcega	2	4	PERSON
Ethiopia	0	1	GPE
18	3	4	CARDINAL
more than 16 million	16	20	CARDINAL
the age of five	22	26	DATE
Tigray	12	13	GPE
Amhara	14	15	GPE
Oromia	16	17	ORG
Wednesday	9	10	DATE
over 100,000	11	13	CARDINAL
Africa	4	5	LOC
Indonesia	10	11	GPE
2003	13	14	DATE
Islamic	18	19	NORP
Nigeria	22	23	GPE
Bolivian	0	1	NORP
the United States	11	14	GPE
U.S.	17

In [26]:
# Test stuff

for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file, n=10000))):
    if i in [802, 736,2297,4823,7902,8206,8633,9004]:
        print(doc)
        print('#############\n')


Representatives of the country's pro-Western coalition say talks scheduled for Monday were canceled after Viktor Yanukovych, the leader of the opposition pro- Russian Party of Regions failed to attend.
#############

Asif Ali Zardari, leader of the Pakistan People's Party, says he wants to set aside the dispute over the Kashmir region so the two countries can focus on other issues, including boosting trade.
#############

Abdul Aziz al-Hakim, the leader of the Supreme Council for the Islamic Revolution in Iraq, told the Washington Post that Iraqis should band together and take up arms to protect their neighborhoods against rampant violence.
#############

Slavkov will lose his position as head of the Bulgarian National Olympic Committee.
#############

Mr. Hakim heads the Shi'ite dominated Supreme Council for the Revolution in Iraq, which has the largest representation in parliament.
#############

A spokesman for the U.S. beef industry, J. Patrick Boyle, head of the American Meat Inst

In [112]:
documents = [doc for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file)))]

#for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file, n=10000))):
#    if i in [4823]:
#        print(doc)
#        print('#############\n')

In [113]:
print(documents[9721])

India's Prime Minister Manmohan Singh has invited Kashmir's pro-independence leader, Yasin Malik, for talks on ending the decades-old dispute over the Himalayan territory.


Read the [section about named entities](https://spacy.io/usage/linguistic-features#section-named-entities) from spaCy&rsquo;s documentation to get some background on this.

## Problem 1: Extract relevant pairs

The first problem that you will have to solve is to identify pairs of entities that are in the &lsquo;is-leader-of&rsquo; relation, as in the example above. There are many ways to do this, but for this lab it suffices to implement the strategy outlined in the section on [Relation Extraction](http://www.nltk.org/book/ch07.html#relation-extraction) in the book by Bird, Klein, and Loper (2009):

* look for all triples of the form $(x, \alpha, y)$ where $x$ and $y$ denote named entities of type *person* and *organisation*, respectively, and $\alpha$ is the intervening text
* write a regular expression to match just those instances of $\alpha$ that express the &lsquo;is-leader-of&rsquo; relation

You can restrict your attention to adjacent pairs of entities &ndash; that is, cases where $x$ precedes $y$ and $\alpha$ does not contain other named entities.

Write a function `extract` that takes an analysed sentence (represented as a spaCy [`Doc`](https://spacy.io/api/doc) object) and yields pairs $(x, y)$ of strings representing entity mentions predicted to be in the &lsquo;is-leader-of&rsquo; relation.

In [71]:
import re

def extract(doc):
    """Extract relevant relation instances from the specified document.
    
    Args:
        doc: The sentence as analysed by spaCy.
    Yields:
        Pairs of strings representing the extracted relation instances.
    """
    # TODO: Replace the following line with your own code
    reg = r'.*\b((head|leader|minister|boss|CEO|chief) of|leads|heads)\b'
    regex = re.compile(reg)
    for i in range(len(doc.ents)-1):
        start,end = doc.ents[i].end,doc.ents[i+1].start
        if(regex.match(str(doc[start:end])) != None and doc.ents[i].label_ == 'PERSON' and doc.ents[i+1].label_ == 'ORG'):
            #print(str(doc[start:end]))
            yield (doc.ents[i].text,doc.ents[i+1].text)
        
    #print('####################')
    #return tm2.extract(doc)

In [33]:
# Some random stuff

for i,doc in enumerate(nlp.pipe(tm2.read_data(data_file, n=10))):
    extract(doc)

The following cell shows how your function is supposed to be used. The code prints out the extracted pairs for the first 1,000&nbsp;sentences in the data. It additionally numbers each pair with the sentence identifier.

In [72]:
for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file, n=10000))):
    for person, org in extract(doc):
        print("{}\t{}\t{}".format(i, person, org))

512	Aung San Suu Kyi	the National League for Democracy
736	Viktor Yanukovych	Russian Party
802	Asif Ali Zardari	the Pakistan People's Party
2297	Abdul Aziz al-Hakim	the Supreme Council for the Islamic Revolution in Iraq
4567	Bush	the U.S. Justice Department
8206	J. Patrick Boyle	the American Meat Institute
8633	Ali Rodriguez	Petroleos de Venezuela
9004	Joschka Fischer	the Green Party
9021	Hassan Halemi	Kabul University
9731	Pedro Esquisabel Urtuzaga	ETA


Once you feel confident that your `extract` function does what it is supposed to do, execute the following cell to extract the entities from the full data set. Note that this will take several minutes (remember that we are processing 62k sentences).

In [73]:
extracted = set()
for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file))):
    for person, org in extract(doc):
        extracted.add((i, person, org))
    print('\rProcessed {} sentences ...'.format(i+1), end='', flush=True)
print(' done')

Processed 62010 sentences ... done


After executing the above cell, all extracted id-string-string triples are in the set `extracted`. The code in the next cell will print the first 10&nbsp;triples in this set.

In [80]:
for i, person, org in sorted(extracted)[:20]:
    print("{}\t{}\t{}".format(i, person, org))
    
print('\n')
for i, person, org in sorted(gold)[:20]:
    print("{}\t{}\t{}".format(i, person, org))

512	Aung San Suu Kyi	the National League for Democracy
736	Viktor Yanukovych	Russian Party
802	Asif Ali Zardari	the Pakistan People's Party
2297	Abdul Aziz al-Hakim	the Supreme Council for the Islamic Revolution in Iraq
4567	Bush	the U.S. Justice Department
8206	J. Patrick Boyle	the American Meat Institute
8633	Ali Rodriguez	Petroleos de Venezuela
9004	Joschka Fischer	the Green Party
9021	Hassan Halemi	Kabul University
9731	Pedro Esquisabel Urtuzaga	ETA
10395	Younus Khalis	Taleban
10441	Mohamed ElBaradei	the International Atomic Energy Agency
11259	Joseph Domenech	U.N.
13043	David Petraeus	U.S. Central Command
14855	Gerry Adams	IRA
15203	Joseph Kony	Resistance Army
15906	Chen Shui-bian	Democratic Progressive Party
19681	Jerry Adams	IRA
19865	Benedict	the Catholic Church
20273	Gerry Adams	IRA


802	Ali Zardari	Pakistan People 's Party
2297	Abdul Aziz al-Hakim	Supreme Council
4823	Slavkov	Bulgarian National Olympic Committee
7902	Mr. Hakim	Supreme Council
8206	J. Patrick Boyle	American M

## Problem 2: Evaluate your system

You now have an extractor, but how good is it? Your task now is to write code that computes the precision, recall, and F1 measure of your extractor relative to the gold standard.

In [91]:
def evaluate(reference, predicted):
    """Print out the precision, recall, and F1 for the id-entity-entity
    triples in the set `predicted`, given the triples in the reference set.
    
    Args:
        reference: The reference set of triples.
        predicted: The set of predicted triples.
    Returns:
        Nothing, but prints out precision, recall, and F1.
    """
    # TODO: Replace the next line with your own code
    common_elements = reference.intersection(predicted)
    p = len(common_elements)/len(predicted)
    r = len(common_elements)/len(reference)
    f = 2*p*r/(p+r)
    print('P: ', p)
    print('R: ', r)
    print('F:', f)
    #tm2.evaluate(reference, predicted)

The next cell shows how your function is intended to be used, as well as the suggested output format.

In [92]:
evaluate(gold, extracted)

P:  0.0684931506849315
R:  0.10869565217391304
F: 0.08403361344537814


In [88]:
tm2.evaluate(gold, extracted)

P: 6.85%
R: 10.87%
F: 8.40%


## Problem 3: Entity resolution

Looking at the results of your quantitative evaluation, you will realise that your extractor (probably) does a rather poor job in matching the gold standard. One reason for this is that the NLP preprocessing is not perfect (spaCy was not trained on the annotations in the Groningen Meaning Bank), and that the approach of using regular expressions for relation extraction is rather naive.

Another reason however is that the current version of your system does not include a component for *entity resolution*. To give an example, your system does not realise that the strings `David Petraeus` and `General David Petraeus` refer to the same entity.

While writing a &lsquo;real&rsquo; entity resolver is beyond the scope of this assignment, we ask you to &lsquo;fake&rsquo; such a resolver. More specifically, you should implement a function `normalise` that takes an entity mention (a string) as its input and rewrites it to the form used in the gold standard. While this is &lsquo;cheating&rsquo;, it allows you to assess the performance of a more realistic system, and helps to illustrate that information extraction can be very domain-specific.

The following cell contains skeleton code for the `normalise` function.

In [134]:
def normalise(text):
    for (i,e1,e2) in gold:
        if (text in e1) or (e1 in text):
            return e1
        elif (text in e2) or (e2 in text):
            return e2
    return text

The next cell shows how `normalise` is intended to be used. Each triple in the set `extracted` is transformed by feeding the two entity mentions into the `normalise` function. The normalised triples are then added to a new set `extracted_normalised`.

In [95]:
extracted_normalised = set()
for triple in extracted:
    extracted_normalised.add((triple[0], normalise(triple[1]), normalise(triple[2])))

To pass the assignment, you should add enough normalisation rules to `normalise` to achieve a recall of at least 50%.

In [96]:
evaluate(gold, extracted_normalised)

P:  0.3561643835616438
R:  0.5652173913043478
F: 0.43697478991596633


## Problem 4: Error analysis

Your last task in this assignment is to do a qualitative error analysis of your information extraction system. You can do this either by writing code or by manual work (inspecting the data file), or mix the two strategies. You can also use the visualisation tools provided by spaCy. For example, the following code cell visualises the output of the named entity recogniser for the given input sentence:

In [97]:
from spacy import displacy

sentence = u'Slavkov will lose his position as head of the Bulgarian National Olympic Committee.'

displacy.render(nlp(sentence), style='ent', jupyter=True)

In any case, you should enter your pairs in the provided text boxes. Use the triple format shown above, where for each pair you also specify the sentence id (line number in the data file) from which the instance was extracted.

### Recall-related errors (false negatives)

By tuning the `normalise` function, you can deal with some of the recall-related mistakes that your system makes. Other recall-related errors cannot be fixed in this way. To illustrate this, find at least 5&nbsp;entity pairs in the gold standard that your system still does not identify correctly, and enter them into the text box below. For each example, provide a brief explanation of what goes wrong. Try to find examples that illustrate different types of errors.

In [135]:
diff1 = gold.difference(extracted_normalised)
print('sentence_id \t entity1 \t entity2\n')
for e in sorted(diff1)[:5]:
    print(e[0],'\t\t',e[1],'\t',e[2])
    

sentence_id 	 entity1 	 entity2

802 		 Ali Zardari 	 Pakistan People 's Party
4823 		 Slavkov 	 Bulgarian National Olympic Committee
7902 		 Mr. Hakim 	 Supreme Council
11021 		 Khalaf 	 al-Qaida
13043 		 David Petraeus 	 U.S. Central Command
General David Petraeus 60729
General David Petraeus


In [115]:
for i in [802,4823,7902,11021,13043]:
    displacy.render(nlp(documents[i].text), style='ent', jupyter=True)

*Enter your explanations here*

Sentence 802: Here our normaliser fails. This is because the entity "the Pakistan People's Party" is represented as "Pakistan People 's Party" in the gold standard, i.e. it has a space before the '. Our normaliser can not handle this.

Sentence 4823: Slavkov is considered a Geopolotical entity in our system, but is a PERSON in the gold standard

Sentence 7902: In our system there is an entity between "Mr. Hakim" and "Supreme council..". Our regular expression only considers text between two adjacent entities.

Sentence 11021: Same issue as 4823

Sentence 13043: The entity "General David Petraeus" exists in this form in doc 60729, our normaliser finds this before the expected "David Petraeus".

### Precision-related errors (false positives)

Next, provide at least 5 entity pairs that represent false positives of your system. Explain what goes wrong.

In [111]:
diff1 = extracted_normalised.difference(gold)
print('sentence_id \t entity1 \t entity2\n')
for e in sorted(diff1)[:5]:
    print(e[0],'\t\t',e[1],'\t',e[2])

sentence_id 	 entity1 	 entity2

512 		 Aung San Suu Kyi 	 the National League for Democracy
736 		 Viktor Yanukovych 	 Russian Party
802 		 Ali Zardari 	 the Pakistan People's Party
4567 		 Bush 	 the U.S. Justice Department
9021 		 Hassan Halemi 	 Kabul University


In [136]:
for i in [512,736,802,4567,9021]:
    displacy.render(nlp(documents[i].text), style='ent', jupyter=True)

*Enter your explanations here*

### Incompleteness of the gold standard

You may have noticed that some of your system&rsquo;s false positives are actually &lsquo;correct&rsquo;. This can happen because, while each entity pair in the gold standard has been manually checked for correctness, no check has been made that the gold standard contains all relevant pairs. Find at least 5&nbsp;entity pairs in the data that are valid instances of the &lsquo;is-leader-of&rsquo; relation (according to your subjective judgement) but that are not contained in the gold standard.

Did you find any examples that you did not find when looking for false positives?

This is the end of the assignment.